<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Darvas_Box_Trend_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  openpyxl
!pip install tradingview-screener==2.5.0
!pip install tqdm

# Gerekli kütüphaneler
import pandas as pd
import numpy as np
from tqdm import tqdm
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import openpyxl

warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimi sayısını seçmesini sağlayan fonksiyon
def get_interval_count_choice():
    print("Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?")
    print("1: Tek Zaman Dilimi")
    print("2: İki Zaman Dilimi")

    choice = input("Seçiminizi yapın (1-2): ")

    if choice == '1':
        return 1
    elif choice == '2':
        return 2
    else:
        print("Geçersiz seçim, varsayılan olarak iki zaman dilimi seçildi.")
        return 2

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(count=2):
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimi(lerini) seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice_1 = input("Birinci zaman dilimini seçin (1-9): ")
    interval_1 = intervals.get(choice_1, ('4 Saat', Interval.in_4_hour))

    if count == 2:
        choice_2 = input("İkinci zaman dilimini seçin (1-9): ")
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily))
        return [interval_1, interval_2]

    return [interval_1]

# Darvas Box stratejisini uygular
def darvas_box_strategy(data):
    df = data.copy()
    df['High_Max'] = df['High'].rolling(window=20).max()  # 20 bar periyodu,
    df['Low_Min'] = df['Low'].rolling(window=20).min()    # 20 bar periyodu,
    df['Darvas_Box_Up'] = df['Close'] > df['High_Max'].shift(1)  # Üst sınır kırılımı (alım sinyali)
    df['Darvas_Box_Down'] = df['Close'] < df['Low_Min'].shift(1)  # Alt sınır kırılımı (satış sinyali)
    return df

# TradingView API'ye bağlan
tv = TvDatafeed()

# BIST hisse senetlerini al
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

# Zaman dilimi sayısını seçme
interval_count = get_interval_count_choice()

# Kullanıcının seçimine göre zaman dilimlerini al
intervals = get_interval_choice(interval_count)

# Sinyalleri depolamak için DataFrame'ler oluşturma
df_darvas_up = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Sinyal Türü', 'Zaman Dilimi', 'Sinyal Zamanı'])
df_darvas_down = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Sinyal Türü', 'Zaman Dilimi', 'Sinyal Zamanı'])

# Seçilen zaman dilimlerine göre analiz yapar
for interval_data in intervals:
    interval_name = interval_data[0]
    interval = interval_data[1]

    # Hisseler üzerinde ilerleme çubuğu ekleyerek döngüyü çalıştırma
    for i in tqdm(range(len(Hisseler)), desc=f" {interval_name}", unit="hisse" ):
        try:
            data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()
            darvas_data = darvas_box_strategy(data)
            darvas_data['datetime'] = pd.to_datetime(darvas_data['datetime'])
            darvas_data.set_index('datetime', inplace=True)
            last_signal = darvas_data.iloc[-1]
            Darvas_Up = last_signal['Darvas_Box_Up']
            Darvas_Down = last_signal['Darvas_Box_Down']
            Last_Price = last_signal['Close']
            Signal_Time = last_signal.name  # Sinyal zamanı

            if Darvas_Up:
                df_darvas_up.loc[len(df_darvas_up)] = [Hisseler[i], Last_Price, 'Darvas Box Yukarı', interval_name, Signal_Time]
                print(f"Tamamlandı: {Hisseler[i]} - {interval_name} - Darvas Box Yukarı - Sinyal Zamanı: {Signal_Time}")
            elif Darvas_Down:
                df_darvas_down.loc[len(df_darvas_down)] = [Hisseler[i], Last_Price, 'Darvas Box Aşağı', interval_name, Signal_Time]
                print(f"Tamamlandı: {Hisseler[i]} - {interval_name} - Darvas Box Aşağı - Sinyal Zamanı: {Signal_Time}")
        except Exception as e:
            print(f"Error processing {Hisseler[i]}: {e}")

# Darvas Box Yukarı ve Aşağı sinyallerini tek bir tabloya ekleyelim
df_all_signals = pd.concat([df_darvas_up, df_darvas_down]).reset_index(drop=True)

# Ekranda tablo olarak gösterme
print("Darvas Box Yukarı Sinyalleri:")
print(df_darvas_up)
print("\nDarvas Box Aşağı Sinyalleri:")
print(df_darvas_down)

# Excel dosyasına kaydetme
excel_file = 'darvas_box_signals.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    df_darvas_up.to_excel(writer, sheet_name='Darvas Box Yukarı', index=False)
    df_darvas_down.to_excel(writer, sheet_name='Darvas Box Aşağı', index=False)
    df_all_signals.to_excel(writer, sheet_name='Tüm Sinyaller', index=False)

print(f"\nDarvas Box yukarı ve aşağı sinyalleri '{excel_file}' dosyasına kaydedildi.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-s5sj6e_6
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-s5sj6e_6
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?
1: Tek Zaman Dilimi
2: İki Zaman Dilimi
Seçiminizi yapın (1-2): 2
Lütfen zaman dilimi(lerini) seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Birinci zaman dilimini seçin (1-9): 7
İkinci zaman dilimini seçin (1-9): 6


 Günlük:   3%|▎         | 15/585 [00:08<05:56,  1.60hisse/s]

Tamamlandı: AKCNS - Günlük - Darvas Box Yukarı - Sinyal Zamanı: 2025-01-24 06:00:00


 Günlük:   3%|▎         | 20/585 [00:10<04:42,  2.00hisse/s]

...

Tamamlandı: ULKER - 4 Saat - Darvas Box Yukarı - Sinyal Zamanı: 2025-01-24 14:00:00


 4 Saat: 100%|█████████▉| 583/585 [07:27<00:01,  1.19hisse/s]

Tamamlandı: ZPBDL - 4 Saat - Darvas Box Yukarı - Sinyal Zamanı: 2025-01-24 14:00:00


 4 Saat: 100%|██████████| 585/585 [07:28<00:00,  1.30hisse/s]

Darvas Box Yukarı Sinyalleri:
   Hisse Adı  Son Fiyat        Sinyal Türü Zaman Dilimi       Sinyal Zamanı
0      AKCNS     206.30  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
1      AKGRT       8.23  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
2      ASELS      85.15  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
3      ASTOR     128.00  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
4      BINBN     101.50  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
5      BSOKE      12.83  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
6      CIMSA      55.50  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
7      DCTTR      41.50  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
8      DGATE      52.40  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
9       DOCO    7087.50  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
10     EDATA      14.74  Darvas Box Yukarı       Günlük 2025-01-24 06:00:00
11     FRIGO       8.74  Darvas Box Yukarı       Günlük 20